In [1]:
import os

In [2]:
%pwd

'e:\\Karan_Bais\\python\\Ml\\Projects\\Text_summary\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\Karan_Bais\\python\\Ml\\Projects\\Text_summary'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir : Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from  Text_summary.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from Text_summary.utils.common import read_yaml,create_directories

In [15]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(path_to_yaml=config_filepath)
        self.params = read_yaml(path_to_yaml=params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(

            root_dir = Path(config.root_dir),
            source_URL = config.source_URL,
            local_data_file = Path(config.local_data_file),
            unzip_dir = Path(config.unzip_dir),
        )
        
        return data_ingestion_config

In [16]:
import os
import urllib.request as req
import zipfile
from Text_summary.logging import logger
from Text_summary.utils.common import get_size_in_KB

In [17]:
class DataIngestion:
    def __init__(self, data_ingestion_config: DataIngestionConfig):
        self.config = data_ingestion_config
    
    def download_data(self):
        if not os.path.exists(self.config.local_data_file):
            filename,header = req.urlretrieve(
                url = self.config.source_URL,
                filename= self.config.local_data_file
            )
            logger.info(f"Downloaded {self.config.local_data_file} {get_size_in_KB(Path(self.config.local_data_file))}")
        else:
            logger.info(f"{self.config.local_data_file} already exists")
    
    def extract_data(self):
        unzip_path = self.config.unzip_dir

        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            logger.info(f"Extracted {self.config.local_data_file} to {unzip_path}")


In [18]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(data_ingestion_config = data_ingestion_config)
    data_ingestion.download_data()
    data_ingestion.extract_data()
except Exception as e:
    raise e

[2024-04-28 17:48:30,819]: INFO: common: yaml file: config\config.yaml loaded]
[2024-04-28 17:48:30,821]: INFO: common: yaml file: params.yaml loaded]
[2024-04-28 17:48:30,821]: INFO: common: Created directory: artifacts]
[2024-04-28 17:48:30,822]: INFO: common: Created directory: artifacts/data_ingestion]
[2024-04-28 17:48:30,823]: INFO: 2362470880: artifacts\data_ingestion\data.zip already exists]
[2024-04-28 17:48:30,945]: INFO: 2362470880: Extracted artifacts\data_ingestion\data.zip to artifacts\data_ingestion]
